<a href="https://colab.research.google.com/github/maysis175/text-processing/blob/master/text_processing_task2B_k10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 8.2MB/s 
     |████████████████████████████████| 552kB 49.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=24b3ae2ecd3ea56e3f3ccbb4565366fd6457e6c3c64f3af8d347126c537276d2
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=7a1528ba92d2d76f67315b2296f8020df64a526d36013738a310f5eecfbfd375
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [2]:
#nltk使ったことない人は、pip install してから対話環境等で以下のdataset等をダウンロードしてください
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
#dataset読み込み
from nltk.corpus import reuters as corpus

In [4]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears  
among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They  
told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And  
lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long -  
run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will  
impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to  
stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at  
10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports 

In [5]:
#全document数
len(corpus.fileids())

10788

In [6]:
#全documentで学習する場合
docs=[corpus.words(fileid) for fileid in corpus.fileids()]

print(docs[:5])
print("num of docs:", len(docs))

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', ...], ['CHINA', 'DAILY', 'SAYS', 'VERMIN', 'EAT', '7', '-', ...], ['JAPAN', 'TO', 'REVISE', 'LONG', '-', 'TERM', ...], ['THAI', 'TRADE', 'DEFICIT', 'WIDENS', 'IN', 'FIRST', ...], ['INDONESIA', 'SEES', 'CPO', 'PRICE', 'RISING', ...]]
num of docs: 10788


### 前処理

In [7]:
# stopwordsリストの作成
en_stop = nltk.corpus.stopwords.words('english')

en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<",")."]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12","86","1986","1987","000"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
         +en_stop

In [8]:
from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

# Cleaning
import re

def cleaning_text(text): 
    pattern = "[0-9]+"
    text = re.sub(pattern, '', text)
    return text

def preprocess_word(word, stopwordset):
    word = word.lower()

    if word in [",","."]:
        return None

    if word in stopwordset:
        return None

    word = cleaning_text(word)

    lemma = wn.morphy(word)
    if lemma is None:
        return word
    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [9]:
#before
print(docs[0][:25]) 

#after
print(preprocess_documents(docs)[0][:25])

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears']
['asian', 'exporter', 'fear', 'damage', 'japan', 'rift', 'mounting', 'trade', 'friction', 'japan', 'raise', 'fear', 'among', 'many', 'asia', 'exporting', 'nation', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen']


### LDA

In [10]:
import gensim
from gensim import corpora

#documentを，gensim LDAが読み込めるデータ構造にする

#辞書の作成
dictionary = corpora.Dictionary(preprocess_documents(docs))
#コーパスの作成
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

#Dictionary:gensimにおける辞書クラス
#token2id属性には単語と辞書IDとの対応が格納される

print(dictionary.token2id)

{'': 0, '>.': 1, 'able': 2, 'accounting': 3, 'action': 4, 'advantage': 5, 'allege': 6, 'allow': 7, 'also': 8, 'american': 9, 'among': 10, 'analyst': 11, 'april': 12, 'asia': 13, 'asian': 14, 'ask': 15, 'association': 16, 'australia': 17, 'australian': 18, 'avow': 19, 'await': 20, 'aware': 21, 'barrier': 22, 'beef': 23, 'beyond': 24, 'big': 25, 'block': 26, 'boost': 27, 'broker': 28, 'budget': 29, 'business': 30, 'businessmen': 31, 'button': 32, 'call': 33, 'canberra': 34, 'capel': 35, 'capital': 36, 'centre': 37, 'chairman': 38, 'chief': 39, 'co': 40, 'coal': 41, 'commercial': 42, 'complete': 43, 'concern': 44, 'conflict': 45, 'continue': 46, 'correspondent': 47, 'cost': 48, 'could': 49, 'country': 50, 'curb': 51, 'cut': 52, 'damage': 53, 'day': 54, 'defuse': 55, 'democratic': 56, 'deputy': 57, 'despite': 58, 'deterioration': 59, 'diplomatic': 60, 'director': 61, 'disadvantage': 62, 'dispute': 63, 'domestically': 64, 'due': 65, 'economic': 66, 'economy': 67, 'effort': 68, 'electric': 6

In [11]:
#before
print([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]])

#after
print(dictionary.doc2bow([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]]))

#これを全文書の全文に適用したのがcorpus_

['asian', 'exporters', 'fear', 'damage', 'from', 'u', '.', 's', '.-', 'japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'u', '.', 's', '.', 'and', 'japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.']
[(10, 1), (13, 1), (14, 1), (31, 1), (49, 1), (53, 2), (66, 1), (79, 1), (82, 1), (83, 1), (90, 1), (110, 1), (114, 2), (136, 1), (151, 1), (192, 1), (203, 1), (204, 1), (250, 1)]


#### 学習

In [12]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=10,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

#### パラメータ確認

In [13]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.364*"" + 0.051*"loss" + 0.025*"profit" + 0.023*"qtr" + 0.022*"rev" + 0.014*"note" + 0.013*"inc" + 0.012*"oper" + 0.011*"include" + 0.011*"sales" + 0.010*"th" + 0.010*"shrs" + 0.010*"avg" + 0.008*"gain" + 0.008*"corp"')
(1, '0.015*"dollar" + 0.011*"market" + 0.010*"bank" + 0.010*"currency" + 0.008*"" + 0.008*"exchange" + 0.007*"rates" + 0.007*"yen" + 0.007*"trade" + 0.007*"would" + 0.006*"japan" + 0.006*"economic" + 0.006*"economy" + 0.006*"dealer" + 0.005*"growth"')
(2, '0.016*"" + 0.013*"inc" + 0.011*"corp" + 0.009*"group" + 0.009*"buy" + 0.008*"sell" + 0.008*"stock" + 0.008*"stake" + 0.007*"unit" + 0.007*"co" + 0.007*"acquire" + 0.007*"ltd" + 0.007*"offer" + 0.006*"common" + 0.005*"firm"')
(3, '0.030*"" + 0.009*"offer" + 0.009*"corp" + 0.008*"earnings" + 0.008*"first" + 0.007*"quarter" + 0.007*"inc" + 0.007*"group" + 0.007*"would" + 0.007*"expect" + 0.006*"stock" + 0.006*"cyclops" + 0.006*"result" + 0.005*"report" + 0.005*"also"')
(4, '0.136*"" + 0.016*"january" + 0.016*"febru

In [14]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(0, 0.036968198), (1, 0.18034929), (2, 0.060765397), (6, 0.7204605)]
document ID 1:[(4, 0.18519533), (5, 0.27188197), (6, 0.53042024)]
document ID 2:[(1, 0.031017091), (2, 0.113743074), (4, 0.2837284), (5, 0.30546027), (6, 0.26154578)]
document ID 3:[(1, 0.049247418), (4, 0.69763535), (7, 0.24558832)]
document ID 4:[(4, 0.13623156), (5, 0.7588771), (6, 0.09781912)]
document ID 5:[(2, 0.091147676), (6, 0.90170777)]
document ID 6:[(1, 0.13277586), (5, 0.10393234), (6, 0.45145318), (7, 0.30929568)]
document ID 7:[(5, 0.9709609)]
document ID 8:[(0, 0.040535178), (1, 0.030264119), (2, 0.21941477), (5, 0.7010889)]
document ID 9:[(0, 0.02657426), (1, 0.47124305), (2, 0.094482444), (3, 0.2706709), (6, 0.10606937), (8, 0.01635815)]


In [15]:
#documentのcategory
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [16]:
n=0

#n番目のdocumentのトピック分布
print(ldamodel.get_document_topics(corpus_[n]))

#n番目のdocumentのcategory
print(categories[n])

#n番目のdocumentの生の文章
print(" ".join(docs[n]))

[(0, 0.036968566), (1, 0.1803504), (2, 0.060763393), (6, 0.720461)]
['trade']
ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long - run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for

#### 可視化

In [17]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

#全documentを学習に用いた場合結構時間がかかる(20min~)
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [18]:
#上で可視化したモデルをgoogle drive上にsaveできる

pyLDAvis.save_html(lda_display,'vis.html')

In [19]:
!ls

sample_data  vis.html
